In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from includes.helpers import get_function_train_test_set
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#Prep Data
X_train, X_test, y_train, y_test = get_function_train_test_set()

In [ ]:
print(tf.config.list_physical_devices('CPU'))
print(tf.config.list_physical_devices('GPU'))

In [ ]:
print(X_train.size)
print(X_train.columns)

In [ ]:
model = Sequential([
  Input(shape=(X_train.shape[1],)),
  Dense(128, activation='relu'),
  Dropout(0.3),
  Dense(64, activation='relu'),
  Dropout(0.2),
  Dense(32, activation='relu'),
  Dropout(0.1),
  Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
from includes.constants import FN_TF_EPOCHS, FN_TF_BATCH

# Train the model
MODEL_EPOCHS = FN_TF_EPOCHS
MODEL_BATCH = FN_TF_BATCH
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=MODEL_EPOCHS, batch_size=MODEL_BATCH, validation_data=(X_test, y_test),callbacks=[early_stopping])

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%\nLoss: {loss}')


In [ ]:
model.save(f'FNpredict_TF_{str(MODEL_EPOCHS)}_{str(MODEL_BATCH)}_{round(accuracy*100)}.keras')  # Saved in HDF5 format, or use .tf for SavedModel format
# model.save(f'testFNpredict_TF_{str(MODEL_EPOCHS)}_{str(MODEL_BATCH)}_{round(accuracy*100)}MAC.keras')  # Saved in HDF5 format, or use .tf for SavedModel format

### Pre-plots

In [ ]:
from sklearn.metrics import classification_report

# Predict on test data
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Generate the report
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import cohen_kappa_score, matthews_corrcoef

# Ground truth and predictions
y_true = y_test  # Actual labels
y_pred = (model.predict(X_test) > 0.5).astype(int).ravel()  # Binary predictions

# Calculate Cohen's Kappa
kappa = cohen_kappa_score(y_true, y_pred)
print(f"Cohen's Kappa: {kappa:.4f}")

# Ground truth and predictions
y_true = y_test  # Actual labels
y_pred = (model.predict(X_test) > 0.5).astype(int).ravel()  # Binary predictions

# Calculate MCC
mcc = matthews_corrcoef(y_true, y_pred)
print(f"Matthews Correlation Coefficient: {mcc:.4f}")



# Plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Generate Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Generate Classification Report
report = classification_report(y_test, y_pred, target_names=["Class 0", "Class 1"])
print("Classification Report:\n")
print(report)

In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC Curve
y_pred_proba = model.predict(X_test).ravel()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line (random chance)
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

# Compute Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
average_precision = average_precision_score(y_test, y_pred_proba)

# Plot Precision-Recall Curve
plt.figure()
plt.plot(recall, precision, label=f'Precision-Recall Curve (AP = {average_precision:.2f})')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.show()

In [ ]:
# Calculate Cohen's Kappa and MCC
kappa = cohen_kappa_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

# Bar Graph
plt.bar(["Cohen's Kappa", "MCC"], [kappa, mcc], color=['blue', 'orange'])
plt.title('Cohen\'s Kappa and MCC')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.show()

In [ ]:
import numpy as np

# Bar plot of predicted classes
unique, counts = np.unique(y_pred, return_counts=True)
plt.bar(unique, counts, color=['green', 'red'])
plt.title('Distribution of Predictions')
plt.xlabel('Predicted Class')
plt.ylabel('Count')
plt.show()

In [ ]:
from sklearn.calibration import calibration_curve

# Calibration Curve
prob_true, prob_pred = calibration_curve(y_test, y_pred_proba, n_bins=10)

plt.plot(prob_pred, prob_true, marker='o')
plt.plot([0, 1], [0, 1], 'k--')  # Perfect calibration line
plt.title('Calibration Curve')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.show()

#### SHAP Stuff

In [ ]:
import shap
import pickle

In [ ]:
# Explain model predictions using SHAP
explainer = shap.Explainer(model, X_test)
shap_values = explainer(X_test)

In [ ]:
def save_object(obj):
  try:
    with open("data.pickle", "wb") as f:
      pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
  except Exception as ex:
    print("Error during pickling object (Possibly unsupported):", ex)

In [ ]:
save_object(shap_values)

**Resume SHAP**

In [ ]:
shap_values = pickle.load(open("data.pickle", "rb"))

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar",title="Feature Importance DNN")
shap.plots.beeswarm(shap_values)
shap.summary_plot(shap_values, plot_type='violin')

In [ ]:
shap.initjs()
shap.plots.heatmap(shap_values[:1000])